In [ ]:
!pip install beautifulsoup4 requests pandas matplotlib plotly

In [1]:
import requests
import pandas as pd

In [2]:
url = f'https://graph.instagram.com/me/media?fields=id,caption,timestamp,\
media_type&access_token=IGQWRPcTZA3ejVHZAThTQmd5SDZAlMDBPNjhWQnlsY3pDcWhFZATFzTG5BemVVVkplTHJxWEh3MEdyZAGpNY3g0TVpuekVfMEFjZA3FwNHhMU1N0NndyeFFIRlJfNTVFM3VyTy1jQ1h4bm1wNHlJcG9YY090YXUtTHA1X0kZD'

In [3]:
response = requests.get(url)

In [4]:
if response.status_code == 200:
    data = response.json()
else:
    print("Error making the API request:", response.text)
    data = None

In [5]:
if data:
    df = pd.DataFrame(data['data'])  # Assuming 'data' is a dictionary containing 'data' key with a list of records
else:
    df = pd.DataFrame(columns=['id', 'caption','timestamp','mediatype','username'])


In [ ]:
print(df)

In [7]:
df.to_csv('instagram_data.csv', index=False)

In [ ]:
!pip install scikit-learn keras tensorflow


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, Concatenate, Dot, Activation

In [10]:
training_data = pd.read_csv('training_data.csv')

In [11]:
instagram_data = pd.read_csv('instagram_data.csv')

In [12]:
instagram_data.dropna(subset=['caption'], inplace=True)
X_train = training_data['caption'].astype(str)
y_train = training_data['depression_label']
X_test = instagram_data['caption'].astype(str)

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [14]:
max_len = 100  # Define your maximum sequence length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [15]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = Dense(1, activation='tanh')(lstm_layer)
attention = Activation('sigmoid')(attention)
context = Dot(axes=1)([attention, lstm_layer])
output_layer = Dense(1, activation='sigmoid')(context)

model = Model(inputs=[input_layer], outputs=output_layer)

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=64)

In [18]:
predicted_labels = model.predict(X_test_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 848ms/step


In [19]:
instagram_data['predicted_depression_label'] = predicted_labels.reshape(-1)


In [20]:
depressing_captions = instagram_data[instagram_data['predicted_depression_label'] > 0.9]['caption']


In [ ]:
for caption in depressing_captions:
    print(caption)


In [ ]:
instagram_data